
# 【建立標籤模型 - 分析流程大綱 (範例程式碼)】

---

## 一、[環境的初始設定](#1)

   
## 二、[自然語言處理 - 將文章向量化](#2)

   
## 三、[機器學習建模 - 將文章貼標/分類](#3)


## 四、[模型應用 - 將新文章貼標/分類](#4)


## 五、[使用完畢，請記得停止與SAS的連結](#5)

---
## 一、環境的初始設定<a name="1"/>

In [1]:
import numpy as np
import pandas as pd

# 匯入SAS Viya API - swat
import swat
from swat.render import render_html

# 與SAS Viya建立session
# 遠端環境1的隊伍，請輸入210.65.88.61
# 遠端環境2的隊伍，請輸入210.65.88.63
# 遠端環境3的隊伍，請輸入210.61.148.118
# 遠端環境4的隊伍，請輸入210.65.88.93
# 遠端環境5的隊伍，請輸入210.65.88.66
SAS = swat.CAS('依上述，請輸入您隊伍使用的IP', 5570, '請輸入您隊伍的環境帳號', '請輸入您的第二個密碼')

# 呈現SAS Viya的系統資訊
status = SAS.serverstatus()
status

NOTE: Grid node action status report: 1 nodes, 7 total actions executed.


[About]

 {'CAS': 'Cloud Analytic Services',
  'Copyright': 'Copyright © 2014-2017 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2018-04-29T12:26:01Z',
  'System': {'Hostname': '210-65-88-61',
   'Model Number': 'x86_64',
   'OS Family': 'LIN X64',
   'OS Name': 'Linux',
   'OS Release': '3.10.0-693.21.1.el7.x86_64',
   'OS Version': '#1 SMP Wed Mar 7 19:03:37 UTC 2018'},
  'Version': '3.03',
  'VersionLong': 'V.03.03M0P11082017',
  'license': {'expires': '09Jun2018:00:00:00',
   'gracePeriod': 0,
   'maxCPUs': 16,
   'site': 'SAS INSTITUTE TAIWAN LTD-SAS MINING CHAMPIONSHIP',
   'siteNum': 70236247,
   'warningPeriod': 5}}

[server]

 Server Status
 
    nodes  actions
 0      1        7

[nodestatus]

 Node Status
 
                               name        role  uptime  running  stalled
 0  210-65-88-61.hinet-ip.hinet.net  controller   0.997        0        0

+ Elapsed: 0.00283s, user: 0.0019s, sys: 0.000844s, mem: 0.337mb

In [2]:
# 呈現指定CAS資料館的資料集
SAS.tableinfo(caslib="casuser")

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,SourceName,SourceCaslib,Compressed,Creator,Modifier
0,PTT_GOSSIPING_BLACKLIST,2985,2,0,utf-8,26Apr2018:12:31:29,26Apr2018:12:31:29,26Apr2018:12:31:29,UTF8,1.840365e+09,1.840365e+09,1.840365e+09,1,0,0,PTT_GOSSIPING_BLACKLIST.sashdat,CASUSER(sas10),0,sas10,
1,PTT_GOSSIPING_201803_TEST,100,12,0,utf-8,26Apr2018:12:31:29,26Apr2018:12:31:29,29Apr2018:10:50:45,UTF8,1.840365e+09,1.840365e+09,1.840618e+09,1,0,0,PTT_GOSSIPING_201803_TEST.sashdat,CASUSER(sas10),0,sas10,
2,TARGET_CUSTOMER,16127,17,0,utf-8,26Apr2018:12:31:30,26Apr2018:12:31:30,26Apr2018:12:31:30,UTF8,1.840365e+09,1.840365e+09,1.840365e+09,1,0,0,TARGET_CUSTOMER.sashdat,CASUSER(sas10),0,sas10,
3,PTT_GOSSIPING_201803_TRAINING,800,12,0,utf-8,26Apr2018:12:31:32,26Apr2018:12:31:32,29Apr2018:10:03:35,UTF8,1.840365e+09,1.840365e+09,1.840615e+09,1,0,0,PTT_GOSSIPING_201803_TRAINING.sashdat,CASUSER(sas10),0,sas10,
4,PTT_TRAINING,800,22,0,utf-8,29Apr2018:10:18:25,29Apr2018:10:18:25,29Apr2018:10:18:30,UTF8,1.840616e+09,1.840616e+09,1.840616e+09,1,0,0,,,0,sas10,


In [3]:
# 載入SAS Viya的分析套件
SAS.loadactionset("textMining")   # 自然語言處理
SAS.loadactionset("autotune")     # 支援自動最佳化模型參數的機器學習模型
SAS.loadactionset("astore")       # SAS開發的高效模型格式 (binary-format and in-memory)

NOTE: Added action set 'textMining'.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'astore'.


[actionset]

 'astore'

+ Elapsed: 0.00114s, user: 3.1e-05s, sys: 0.00107s, mem: 0.195mb

In [4]:
# 瀏覽已載入的SAS Viya的分析套件
SAS.help()

NOTE: Available Action Sets and Actions:
NOTE:    accessControl
NOTE:       assumeRole - Assumes a role
NOTE:       dropRole - Relinquishes a role
NOTE:       showRolesIn - Shows the currently active role
NOTE:       showRolesAllowed - Shows the roles that a user is a member of
NOTE:       isInRole - Shows whether a role is assumed
NOTE:       isAuthorized - Shows whether access is authorized
NOTE:       isAuthorizedActions - Shows whether access is authorized to actions
NOTE:       isAuthorizedTables - Shows whether access is authorized to tables
NOTE:       isAuthorizedColumns - Shows whether access is authorized to columns
NOTE:       listAllPrincipals - Lists all principals that have explicit access controls
NOTE:       whatIsEffective - Lists effective access and explanations (Origins)
NOTE:       whatCheckoutsExist - Lists checkouts held on an object, its parents, and optionally its children
NOTE:       checkOutObject - Reserves an object (and all of its children) for update by o

,name,description
0,assumeRole,Assumes a role
1,dropRole,Relinquishes a role
2,showRolesIn,Shows the currently active role
3,showRolesAllowed,Shows the roles that a user is a member of
4,isInRole,Shows whether a role is assumed
5,isAuthorized,Shows whether access is authorized
6,isAuthorizedActions,Shows whether access is authorized to actions
7,isAuthorizedTables,Shows whether access is authorized to tables
8,isAuthorizedColumns,Shows whether access is authorized to columns
9,listAllPrincipals,Lists all principals that have explicit access...


In [5]:
# 瀏覽指定SAS Viya的分析套件
SAS.help(actionset= "autotune")

NOTE: Information for action set 'autotune':
NOTE:    autotune
NOTE:       tuneSvm - Automatically adjusts support vector machine parameters to tune a model for minimum error
NOTE:       tuneForest - Automatically adjusts forest parameters to tune a model for minimum error
NOTE:       tuneDecisionTree - Automatically adjusts decision tree parameters to tune a model for minimum error
NOTE:       tuneNeuralNet - Automatically adjusts neural network parameters to tune a model for minimum error
NOTE:       tuneGradientBoostTree - Automatically adjusts gradient boosting tree parameters to tune a model for minimum error
NOTE:       tuneFactMac - Automatically adjusts factorization machine parameters to tune a model for minimum error


,name,description
0,tuneSvm,Automatically adjusts support vector machine p...
1,tuneForest,Automatically adjusts forest parameters to tun...
2,tuneDecisionTree,Automatically adjusts decision tree parameters...
3,tuneNeuralNet,Automatically adjusts neural network parameter...
4,tuneGradientBoostTree,Automatically adjusts gradient boosting tree p...
5,tuneFactMac,Automatically adjusts factorization machine pa...


In [6]:
# 瀏覽指定資料集的前10筆
SAS.CASTable(name= 'PTT_GOSSIPING_201803_TRAINING', caslib= 'casuser').fetch(to=10)  

,uniqueid,TITLE,DATE,AUTHOR,IP,PUSH,BOO,ARROW,URL,CONTENT,international,social_issue
0,1107.0,[新聞] 稱議員質詢吳音寧影片遭「移花接木」,1.836667e+09,bray,發信站 批踢踢實業坊 ptt.cc 來自 140.114.253.29,288.0,22.0,112.0,https://www.ptt.cc/bbs/Gossiping/M.1521019398....,1.媒體來源 中時 2.完整新聞標題 稱議員質詢吳音寧影片遭 移花接木 他還...,0,0
1,1207.0,[新聞] 陸客從六成跌至二成八 故宮：如此觀光結,1.836669e+09,Syd,發信站 批踢踢實業坊 ptt.cc 來自 118.170.72.51,104.0,8.0,28.0,https://www.ptt.cc/bbs/Gossiping/M.1521020811....,陸客從六成跌至二成八 故宮 如此觀光結構才健全 2018 03 14 11 41聯合報...,0,0
2,1307.0,[問卦] 哪部少年漫畫主角不是靠爸？,1.836669e+09,MrBing,發信站 批踢踢實業坊 ptt.cc 來自 27.52.226.159,296.0,37.0,81.0,https://www.ptt.cc/bbs/Gossiping/M.1521020902....,媽的 不是說好少年漫畫要勵志嗎 為什麼我看到的主角都有個威能老爸啊 小傑他爸 開...,0,1
3,1407.0,[新聞] 賣偷拍片自誇年年百人斬 MMA洋腸：台灣,1.836670e+09,gohellloser,發信站 批踢踢實業坊 ptt.cc 來自 118.70.124.34,126.0,23.0,40.0,https://www.ptt.cc/bbs/Gossiping/M.1521022212....,賣偷拍片自誇年年百人斬 MMA洋腸 台灣胖妹超愛我 APPLE 受害人數持續擴大...,0,1
4,1507.0,[ＦＢ] 柯文哲,1.836677e+09,LewisRong,發信站 批踢踢實業坊 ptt.cc 來自 123.193.214.39,113.0,13.0,21.0,https://www.ptt.cc/bbs/Gossiping/M.1521028602....,流標5次 中間一度發生弊案的雙子星大樓 在這個月底要重新上網招標了 經過3年下來的努...,0,0
5,1607.0,Re: [問卦] 有沒有人可以簡單解釋1-10維度意思？,1.836683e+09,jfsu,發信站 批踢踢實業坊 ptt.cc 來自 180.177.15.128,200.0,10.0,26.0,https://www.ptt.cc/bbs/Gossiping/M.1521035061....,引述 andyajacky 你當我錐子 之銘言 幹 我啦 彭于晏 剛在水...,0,0
6,1707.0,Re: [新聞] 北農獲利成長 竟是靠大幅減少員工獎金,1.836686e+09,KingKingCold,發信站 批踢踢實業坊 ptt.cc 來自 116.240.229.34,470.0,63.0,283.0,https://www.ptt.cc/bbs/Gossiping/M.1521038019....,引述 oeibei 紅白 之銘言 1.媒體來源 Yahoo論壇 ...,0,0
7,1807.0,[新聞] 港女命案卡在這點 僅通緝兇嫌！恐永遠,1.836717e+09,HANASUCIA,發信站 批踢踢實業坊 ptt.cc 來自 61.228.155.49,155.0,43.0,88.0,https://www.ptt.cc/bbs/Gossiping/M.1521068927....,1.媒體來源 蘋果 2.完整新聞標題 港女命案卡在這點 僅通緝兇嫌 恐永遠結不了案...,0,0
8,1907.0,[問卦] 鄉民484以為米其林星星只看美味程度？,1.836718e+09,hochengyuan,發信站 批踢踢實業坊 ptt.cc 來自 98.109.153.150,130.0,44.0,104.0,https://www.ptt.cc/bbs/Gossiping/M.1521069784....,小弟有朋友念美國廚藝學院 先前他放假也帶我這個窮留學生去紐約餐廳週吃一星的商業午餐 ...,0,0
9,2007.0,[新聞] 色妹幫口活以為豔遇！2天後恐怖爆發,1.836720e+09,caeasonfb,發信站 批踢踢實業坊 ptt.cc 來自 27.105.238.94,102.0,31.0,38.0,https://www.ptt.cc/bbs/Gossiping/M.1521072055....,東森新聞 色妹幫口活以為豔遇 2天後恐怖爆發 關鍵一句話竟救了他 社會中心 綜合...,0,0


### 【延伸閱讀 - SWAT APIs】
https://sassoftware.github.io/python-swat/sas-tk.html

---
## 二、自然語言處理 - 將文章向量化<a name="2"/>

In [7]:
# 執行自然語言處理，並建立主題模型 (Topic model)
SAS.textMining.tmMine(
    documents = {'name':'ptt_gossiping_201803_training', 'caslib':'casuser'},
    docId     = 'uniqueid',
    text      = 'CONTENT',
    
    language  = 'chinese',
    reduce    = 5,
    entities  = 'STD',
    k         = 20,
    numlabels = 7,
    
    terms     = {'name':'terms', 'replace':True},    
    topics    = {'name':'topics', 'replace':True},
    docPro    = {'name':'doc2topic', 'replace':True},
    savestate = {'name':'model_topic'}) # 將該模型存成astore檔案 (預設資料館為casuser)

NOTE: Wrote 1376618 bytes to the savestate file model_topic.


,casLib,Name,Label,Rows,Columns,casTable
0,CASUSER(sas10),terms,,7011,11,"CASTable('terms', caslib='CASUSER(sas10)')"
1,CASUSER(sas10),doc2topic,,800,21,"CASTable('doc2topic', caslib='CASUSER(sas10)')"
2,CASUSER(sas10),topics,,20,3,"CASTable('topics', caslib='CASUSER(sas10)')"


### 【延伸閱讀 - 自然語言處理套件】
http://go.documentation.sas.com/?cdcId=pgmcdc&cdcVersion=8.11&docsetId=casanpg&docsetTarget=cas-textmining-TblOfActions.htm&locale=zh-TW

In [8]:
# 將訓練好的topic model存成.astore檔案於指定路徑
model_topic = SAS.astore.download(
    rstore = {'caslib' : 'casuser', 'name' : 'model_topic'})

with open('model_topic.astore', 'wb') as file:
    file.write(model_topic['blob'])

In [9]:
# 呈現不重複字詞的資訊，包括詞性、是否為實體、出現次數等
SAS.CASTable(name= "terms").fetch(to=10)  

,_Term_,_Role_,_Attribute_,_Frequency_,_NumDocs_,_Keep_,_Termnum_,_Parent_,_ParentId_,_IsPar_,_Weight_
0,v,N,Alpha,27.0,24.0,Y,1.0,NaN,1.0,,0.532894
1,穿,V,Alpha,46.0,25.0,Y,2.0,NaN,2.0,,0.560512
2,分為,V,Alpha,6.0,6.0,Y,3.0,NaN,3.0,,0.731958
3,導致,V,Alpha,78.0,61.0,Y,4.0,NaN,4.0,,0.400253
4,正是,V,Alpha,8.0,7.0,Y,5.0,NaN,5.0,,0.714844
5,激情,N,Alpha,6.0,5.0,Y,6.0,NaN,6.0,,0.766522
6,阿伯,N,Alpha,12.0,6.0,Y,7.0,NaN,7.0,,0.751527
7,監察院,N,Alpha,13.0,8.0,Y,8.0,NaN,8.0,,0.712007
8,gossippicket,N,Alpha,9.0,9.0,Y,9.0,NaN,9.0,,0.671301
9,三月,nlpDate,Entity,9.0,8.0,Y,10.0,NaN,10.0,,0.694344


In [10]:
# 呈現各個主題維度的字詞組合
SAS.CASTable(name= "topics").fetch(to=10)

,_TopicId_,_Name_,_TermCutOff_
0,1.0,"閔, 兼職, 經濟, 教師, 廈門大學, 大學, 元",0.017
1,2.0,"農, 吳音, 寧, 農委會, 北, 總經理, 陳景",0.016
2,3.0,"the, to, 美國, 瀚, of, in, do",0.016
3,4.0,"麟, 明, 法官, 蕭, 委, 檢察官, 判決書",0.016
4,5.0,"刪除, 貼, 刪, 水桶, 板, 新聞, 完整新聞",0.016
5,6.0,"柯, 文哲, 柯文哲, 收據, 姚立明, 鄭, 弘",0.016
6,7.0,"善款, 花蓮, 紓困, 善政, 善, 捐款, 捐款",0.015
7,8.0,"海邊, 君, 學弟, 經理, 跑步, 當事人, 員工",0.016
8,9.0,"李敖, 李敖, 孟, 蕭, 胡, 粉, 貓",0.016
9,10.0,"男, 女, 她, 警方, 女, 名, 男子",0.017


In [11]:
# 呈現各篇文章映射在主題空間的向量值 (或說座標位置)
SAS.CASTable(name= "doc2topic").fetch(to=10)

,uniqueid,_Col1_,_Col2_,_Col3_,_Col4_,_Col5_,_Col6_,_Col7_,_Col8_,_Col9_,...,_Col11_,_Col12_,_Col13_,_Col14_,_Col15_,_Col16_,_Col17_,_Col18_,_Col19_,_Col20_
0,101.0,0.181256,0.134033,0.144847,0.221429,0.288111,0.136631,0.213564,0.212667,0.292358,...,0.141552,0.196196,0.214502,0.126111,0.437416,0.184308,0.212512,0.227217,0.238420,0.210577
1,110.0,0.057428,0.054730,0.095971,0.067504,0.047014,0.084767,0.927944,0.068018,0.096308,...,0.056953,0.107311,0.134405,0.072653,0.054227,0.105740,0.044985,0.154620,0.072744,0.088358
2,202.0,0.006573,0.079997,0.067393,0.104862,0.085952,0.091662,0.081119,0.065341,0.132456,...,0.100684,0.080922,0.134121,0.851794,0.065829,0.240992,0.081897,0.222336,0.066321,0.224984
3,210.0,0.196894,0.218089,0.133321,0.141799,0.099483,0.136990,0.133179,0.300438,0.191740,...,0.071404,0.107001,0.183990,0.139442,0.080639,0.199009,0.109176,0.186915,0.049760,0.287304
4,302.0,-0.071474,0.185628,0.124848,0.186207,0.193726,0.187078,0.356024,0.280166,0.202221,...,0.109866,0.177420,0.212951,0.176881,0.231472,0.236185,0.199293,0.276018,0.080340,0.415022
5,312.0,-0.125582,0.211122,0.044492,0.072775,0.180409,0.704588,0.047503,0.051628,0.117216,...,0.014710,0.139104,0.098370,0.082516,0.032550,0.287774,0.439210,0.077556,0.048616,0.250490
6,407.0,-0.014210,0.148625,0.107467,0.124753,0.129683,0.208535,0.114787,0.139569,0.159896,...,0.337329,0.111329,0.163947,0.246155,0.584659,0.172800,0.022840,0.359181,0.128003,0.240198
7,415.0,0.143326,0.106677,0.283563,0.115890,0.120773,0.276013,0.088501,0.189456,0.185619,...,0.121078,0.183515,0.279673,0.249637,0.178069,0.237373,0.137577,0.229706,0.211972,0.362012
8,508.0,0.832975,0.048509,-0.121477,0.156855,0.019893,0.096758,0.068916,0.129480,0.097477,...,0.092637,0.046720,0.091753,-0.156729,0.239689,-0.030393,-0.039492,0.011612,0.132712,0.303644
9,516.0,0.541259,0.063315,0.185194,0.210409,0.182939,0.235327,0.253164,0.095157,0.099007,...,0.245672,0.089226,0.264499,0.343697,0.144659,0.266914,0.136332,0.133607,0.191159,-0.013500


In [13]:
# 將指定CAS資料館的資料集存成Python dataframe
tmp = SAS.CASTable(name = 'ptt_gossiping_201803_training', caslib = 'casuser').to_frame()

# 目標變數Y為是否為international(國際要事)；解釋變數X為主題向量
Y = tmp[['uniqueid', 'international']]
X = SAS.CASTable(name= 'doc2topic').to_frame()

# 依uniqueid欄位變數，合併X與Y
data = X.merge(Y, on= 'uniqueid')

# 將Python dataframe匯入指定CAS資料館的資料集
SAS.upload_file(data, casout={'name':'ptt_training', 'caslib': 'casuser', 'replace': 'True'})

NOTE: Cloud Analytic Services made the uploaded file available as table PTT_TRAINING in caslib CASUSER(sas10).
NOTE: The table PTT_TRAINING has been created in caslib CASUSER(sas10) from binary data uploaded to Cloud Analytic Services.


CASTable('PTT_TRAINING', caslib='CASUSER(sas10)')

In [14]:
# 呈現合併結果
SAS.CASTable(name= 'ptt_training', caslib= 'casuser').fetch(to=10)

,uniqueid,_Col1_,_Col2_,_Col3_,_Col4_,_Col5_,_Col6_,_Col7_,_Col8_,_Col9_,...,_Col12_,_Col13_,_Col14_,_Col15_,_Col16_,_Col17_,_Col18_,_Col19_,_Col20_,international
0,101.0,0.181256,0.134033,0.144847,0.221429,0.288111,0.136631,0.213564,0.212667,0.292358,...,0.196196,0.214502,0.126111,0.437416,0.184308,0.212512,0.227217,0.238420,0.210577,0.0
1,110.0,0.057428,0.054730,0.095971,0.067504,0.047014,0.084767,0.927944,0.068018,0.096308,...,0.107311,0.134405,0.072653,0.054227,0.105740,0.044985,0.154620,0.072744,0.088358,0.0
2,202.0,0.006573,0.079997,0.067393,0.104862,0.085952,0.091662,0.081119,0.065341,0.132456,...,0.080922,0.134121,0.851794,0.065829,0.240992,0.081897,0.222336,0.066321,0.224984,0.0
3,210.0,0.196894,0.218089,0.133321,0.141799,0.099483,0.136990,0.133179,0.300438,0.191740,...,0.107001,0.183990,0.139442,0.080639,0.199009,0.109176,0.186915,0.049760,0.287304,0.0
4,302.0,-0.071474,0.185628,0.124848,0.186207,0.193726,0.187078,0.356024,0.280166,0.202221,...,0.177420,0.212951,0.176881,0.231472,0.236185,0.199293,0.276018,0.080340,0.415022,0.0
5,312.0,-0.125582,0.211122,0.044492,0.072775,0.180409,0.704588,0.047503,0.051628,0.117216,...,0.139104,0.098370,0.082516,0.032550,0.287774,0.439210,0.077556,0.048616,0.250490,0.0
6,407.0,-0.014210,0.148625,0.107467,0.124753,0.129683,0.208535,0.114787,0.139569,0.159896,...,0.111329,0.163947,0.246155,0.584659,0.172800,0.022840,0.359181,0.128003,0.240198,1.0
7,415.0,0.143326,0.106677,0.283563,0.115890,0.120773,0.276013,0.088501,0.189456,0.185619,...,0.183515,0.279673,0.249637,0.178069,0.237373,0.137577,0.229706,0.211972,0.362012,0.0
8,508.0,0.832975,0.048509,-0.121477,0.156855,0.019893,0.096758,0.068916,0.129480,0.097477,...,0.046720,0.091753,-0.156729,0.239689,-0.030393,-0.039492,0.011612,0.132712,0.303644,0.0
9,516.0,0.541259,0.063315,0.185194,0.210409,0.182939,0.235327,0.253164,0.095157,0.099007,...,0.089226,0.264499,0.343697,0.144659,0.266914,0.136332,0.133607,0.191159,-0.013500,0.0


---

## 三、機器學習建模 - 將文章貼標/分類 <a name="3"/>

In [15]:
# 建立Gradient boosting模型，並啟動自動最佳化模型參數的機制 (最佳化演算法的預設值為基因演算法)
gb = SAS.autotune.tunegradientboosttree(
        trainOptions = {
            'table'     : {'name':'ptt_training', 'caslib':'casuser'},
            'inputs'    : list(X.columns),
            'target'    : 'uniqueid',
            'nominals'  : 'uniqueid',
            'saveState' : {"name":'model_gb', 'caslib':'casuser', 'replace':'True'}},

        tunerOptions = {
            'objective'         : 'MSE',
            'evaluationHistory' : True,
            'maxEvals'          : 5,
            'maxIters'          : 3,
            'nCrossValFolds'    : 5,
            'seed'              : 1234},

        tuningparameters = [
            {'name': 'nTree', 'lowerBound':100, 'upperBound':100, 'initValue':100},
            {'name': 'm', 'lowerBound':10, 'upperBound':10, 'initValue':10},
            {'name': 'learningRate', 'lowerBound':0.01, 'upperBound':0.03, 'initValue':0.01},
            {'name': 'subSampleRate', 'lowerBound':0.01, 'upperBound':0.03, 'initValue':0.01},
            {'name': 'lasso', 'exclude': True},
            {'name': 'ridge', 'exclude': True}]
)

render_html(gb)

NOTE: Autotune is started for 'Gradient Boosting Tree' model.
NOTE: Autotune option SEARCHMETHOD='GA'.
NOTE: Autotune option MAXEVALS=5.
NOTE: Autotune option MAXTIME=36000 (sec.).
NOTE: Autotune objective is 'Mean Squared Error'.
NOTE: The seed for random partition/fold generation is 1234.
NOTE: Autotune number of parallel evaluations is set to 4, each using 0 worker nodes.
NOTE: Automatic early stopping is activated with STAGNATION=4;  set EARLYSTOP=false to deactivate.
         Iteration       Evals     Best Objective        Time
                 0           1                100       35.46
                 1          10                100      147.99
NOTE: Wrote 852367 bytes to the savestate file model_gb.
NOTE: Due to early stopping, the actual final number of trees used in the model (1) is less than the Autotune selected 'best' value (100).
NOTE: Autotune time is 191.08 seconds.


Gradient Boosting Tree for PTT_TRAINING_AUTOTUNE_29APR2018:20:42:01 
 Descr 
 Value 
 
 
 Number of Trees 80000 
 Distribution 3 
 Learning Rate 0.01 
 Subsampling Rate 0.01 
 Number of Selected Variables (M) 10 
 Number of Bins 20 
 Number of Variables 20 
 Max Number of Tree Nodes 1 
 Min Number of Tree Nodes 1 
 Max Number of Branches 0 
 Min Number of Branches 0 
 Max Number of Levels 2 
 Min Number of Levels 2 
 Max Number of Leaves 1 
 Min Number of Leaves 1 
 Maximum Size of Leaves 8 
 Minimum Size of Leaves 8 
 Random Number Seed 1234 
 
 
 
 
 Evaluation Metric With Gradient Boosting Tree Analytics for PTT_TRAINING_AUTOTUNE_29APR2018:20:42:01 
 Progress 
 Metric 
 
 
 1 0.99875 
 
 
 
 
 Scoring Information Table With Gradient Boosting Tree Analytics for PTT_TRAINING_AUTOTUNE_29APR2018:20:42:01 
 Descr 
 Value 
 
 
 Number of Observations Read 800 
 Number of Observations Used 800 
 Misclassification Error (%) 99.875 
 
 
 
 
 Prediction Error With Gradient Boosting Tree Analytics for PTT_TRAINING_AUTOTUNE_29APR2018:20:42:01 
 TreeID 
 Trees 
 NLeaves 
 MCR 
 LogLoss 
 ASE 
 RASE 
 MAXAE 
 
 
 0 1 1 0.99875 6.691669542 0.0012484818 0.0353338621 0.9987711568 
 
 
 
 
 Tuner Information 
 Parameter 
 Value 
 
 
 Model Type Gradient Boosting Tree 
 Tuner Objective Function Mean Squared Error 
 Search Method GA 
 Maximum Evaluations 5 
 Population Size 10 
 Maximum Iterations 3 
 Maximum Tuning Time in Seconds 36000 
 Validation Type Cross-Validation 
 Num Folds in Cross-Validation 5 
 Log Level 2 
 Seed 1234 
 
 
 
 
 Tuner Results 
 Evaluation 
 NTREE 
 M 
 LEARNINGRATE 
 SUBSAMPLERATE 
 Mean Squared Error 
 Time In Seconds 
 
 
 0 100 10 0.01 0.01 100 35.460490942 
 1 100 10 0.01 0.01 100 0 
 2 100 10 0.0211111111 0.0188888889 100 39.115461 
 3 100 10 0.0144444444 0.0211111111 100 39.26403 
 4 100 10 0.03 0.0122222222 100 36.148616 
 5 100 10 0.0188888889 0.03 100 41.664342 
 6 100 10 0.0122222222 0.0166666667 100 36.901184 
 7 100 10 0.01 0.0233333333 100 37.82676 
 8 100 10 0.0277777778 0.0144444444 100 36.030976 
 9 100 10 0.0255555556 0.01 100 35.648381 
 10 100 10 0.0233333333 0.0255555556 100 39.480855 
 
 
 
 
 Tuner Iteration History 
 Iteration 
 Evaluations 
 Best Objective 
 Time 
 
 
 0 1 100 35.460490942 
 1 10 100 183.45272088 
 
 
 
 
 Tuner Evaluation History 
 Evaluation 
 Iteration 
 NTREE 
 M 
 LEARNINGRATE 
 SUBSAMPLERATE 
 Mean Squared Error 
 Time In Seconds 
 
 
 0 0 100 10 0.01 0.01 100 35.460490942 
 1 1 100 10 0.01 0.01 100 0 
 2 1 100 10 0.0211111111 0.0188888889 100 39.115461 
 3 1 100 10 0.0144444444 0.0211111111 100 39.26403 
 4 1 100 10 0.03 0.0122222222 100 36.148616 
 5 1 100 10 0.0188888889 0.03 100 41.664342 
 6 1 100 10 0.0122222222 0.0166666667 100 36.901184 
 7 1 100 10 0.01 0.0233333333 100 37.82676 
 8 1 100 10 0.0277777778 0.0144444444 100 36.030976 
 9 1 100 10 0.0255555556 0.01 100 35.648381 
 10 1 100 10 0.0233333333 0.0255555556 100 39.480855 
 
 
 
 
 Best Configuration 
 Parameter 
 Name 
 Value 
 
 
 Evaluation Evaluation 1 
 Number of Trees NTREE 100 
 Number of Variables to Try M 10 
 Learning Rate LEARNINGRATE 0.01 
 Sampling Rate SUBSAMPLERATE 0.01 
 Mean Squared Error Objective 100 
 
 
 
 
 Tuner Summary 
 Parameter 
 Value 
 
 
 Initial Configuration Objective Value 100 
 Best Configuration Objective Value 100 
 Worst Configuration Objective Value 100 
 Initial Configuration Evaluation Time in Seconds 35.460490942 
 Best Configuration Evaluation Time in Seconds 35.460490942 
 Number of Improved Configurations 0 
 Number of Evaluated Configurations 10 
 Total Tuning Time in Seconds 191.08215213 
 Parallel Tuning Speedup 2.2013559935 
 
 
 
 
 Tuner Task Timing 
 Task 
 Seconds 
 Percent 
 
 
 Model Training 370.52266502 88.08549002 
 Model Scoring 47.134401798 11.205405961 
 Total Objective Evaluations 417.66756964 99.293392847 
 Tuner 2.972271204 0.7066071531 
 Total CPU Time 420.63984084 100 
 
 
 
 
 Tuner CAS Output Tables 
 CAS Library 
 Name 
 Number of Rows 


### 【延伸閱讀 - 機器學習模型】
http://go.documentation.sas.com/?cdcId=pgmcdc&cdcVersion=8.11&docsetId=casactml&docsetTarget=titlepage.htm&locale=zh-TW

In [16]:
# 將訓練好的gradient boosting存成.astore檔案於指定路徑
model_gb = SAS.astore.download(
    rstore = {'caslib' : 'casuser', 'name' : 'model_gb'})

with open('model_gb.astore', 'wb') as file:
    file.write(model_gb['blob'])

---

## 四、模型應用 - 將新文章貼標/分類<a name="4"/>

In [17]:
# 於指定路徑將訓練好的topic model載入指定的CAS資料館
tmp = open('model_topic.astore', 'rb')
SAS.astore.upload(rstore = {'caslib' : 'casuser', 'name' : 'model_topic', 'replace': 'True'}, store = swat.blob(tmp.read()))

[key]

 '0A10A8E9E7141CE38DA0DEE1FF71A683A0A3B01A'

+ Elapsed: 0.0176s, user: 0.0132s, sys: 0.00406s, mem: 4.4mb

In [18]:
# 於指定路徑將訓練好的gradient boosting載入指定的CAS資料館
tmp = open('model_gb.astore', 'rb')
SAS.astore.upload(rstore = {'caslib' : 'casuser', 'name' : 'model_gb', 'replace': 'True'}, store = swat.blob(tmp.read()))

[key]

 '36857FEAFE9D4A92AE7A5C68A9B482B4CCEF33CE'

+ Elapsed: 0.0114s, user: 0.00815s, sys: 0.00291s, mem: 3.9mb

In [19]:
# 將測試資料集依序使用上述的topic model、gradient boosting作預測 (SAS稱為評分 (score))
SAS.astore.score(
    table  = {'name':'ptt_gossiping_201803_test', 'caslib':'casuser'},
    out    = {'name':'ptt_doc2topic_test'},
    rstore = {'name':'model_topic', 'caslib':'casuser'})

SAS.astore.score(
    table  = {'name':'ptt_doc2topic_test', 'caslib':'casuser'},
    out    = {'name':'outcome_gb_test'},
    rstore = {'name':'model_gb', 'caslib':'casuser'})

,Task,Seconds,Percent
0,Loading the Store,0.000075,0.000228
1,Creating the State,0.026074,0.079307
2,Scoring,0.302621,0.920461
3,Total,0.328771,1.000000


In [20]:
# 將預測結果存成.csv檔案於指定路徑
SAS.CASTable('outcome_gb_test').to_csv('outcome_gb_test.csv', encoding = 'utf-8', index = None)

---

## 五、使用完畢，請記得停止與SAS的連結<a name="1"/>

In [21]:
# 關閉與SAS Viya建立的session
SAS.close()